In [1]:
import mercury as mr
app = mr.App(title="Практическая работа.", description="Расчет дебита скважины")
h = mr.Text(value="20", label="Высота (м):")
fi = mr.Text(value ="0.15", label="Пористость, (ед)")
k = mr.Text(value="1e-15", label="Проницаемость, м^2")
mu = mr.Text(value="1e-3", label="Динамическая вязкость, Па·с")
ro = mr.Text(value="1000", label="Плотность, кг/м^3")
c = mr.Text(value="0.5e-9", label="Сжимаемость, Па^-1")
r_w = mr.Text(value="0.12", label="Радиус скважины, м")
p_0 = mr.Text(value="250e5", label="Начальное давление, Па")
p_zab = mr.Text(value="150e5", label="Забойное давление, Па")

mercury.App

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

In [2]:
import numpy as np
import subprocess
import os
import json
import time

In [ ]:
N = 300

hr = float(h.value)
fir = float(fi.value) 
kr = float(k.value)
mur = float(mu.value)
cr = float(c.value)
ror = float(ro.value)
p_0r = float(p_0.value)
p_zabr = float(p_zab.value)
r_wr = float(r_w.value)

mr.Markdown("Подготовка данных в JSON формате")

base_dir = os.path.dirname(os.getcwd()) 
cpp_core_dir = os.path.join(base_dir, 'src', 'cpp_core')

input_path = os.path.join(cpp_core_dir, 'input_params.json')
exe_path = os.path.join(cpp_core_dir, 'core.exe')
result_path = os.path.join(cpp_core_dir, 'result.json')

try:
    subprocess.run(['taskkill', '/f', '/im', 'core.exe'], 
                   capture_output=True, timeout=5)
except:
    pass

if os.path.exists(result_path):
    os.remove(result_path)

params_dict = {
    "height": hr,
    "porosity": fir,
    "permeability": kr,
    "viscosity": mur,
    "compressibility": cr,
    "density": ror,
    "initial_pressure": p_0r,
    "well_radius": r_wr,
    "bottomhole_pressure": p_zabr,
    "points_count": N
}

with open(input_path, 'w') as f:
    json.dump(params_dict, f, indent=2)

mr.Markdown("Запуск C++ ядра...")

try:

    result = subprocess.run(
        [exe_path], 
        cwd=cpp_core_dir,
        capture_output=True,
        timeout=30
    )
    if result.returncode == 0:
        mr.Markdown("EXE выполнен успешно")

    time.sleep(2)
    
    if os.path.exists(result_path):
        with open(result_path, 'r') as f:
            file_content = f.read()
        
        results_dict = json.loads(file_content)
        
        kf = results_dict["piezometric_coefficient"]
        time_days = np.array(results_dict["time_days"])
        flow_rate_m3_day = np.array(results_dict["flow_rate_m3_day"])
        
        mr.Markdown("Расчет завершен.")
        mr.Markdown(f"""
## Результаты расчета

| Параметр | Значение | Единицы измерения |
|----------|----------|------------------|
| Коэффициент пьезопроводности | {kf:.2e} | м²/с |
| Максимальный дебит | {flow_rate_m3_day.max():.2f} | м³/сут |
| Средний дебит | {flow_rate_m3_day.mean():.2f} | м³/сут |
| Перепад давления | {(p_0r - p_zabr) / 1e5:.1f} | бар |
| Время расчета | {time_days[-1]:.0f} | дней |
""")
        
    else:
        mr.Markdown("Файл result.txt не создан")
        
except Exception as e:
    mr.Markdown(f"Ошибка: {e}")

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    pressure_profile = np.ones(N) * p_zabr
    pressure_profile[0:50] = p_0r
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
        
    ax1.plot(time_days, np.ones(N) * p_0r / 1e5, '--b', label='Пластовое давление')
    ax1.plot(time_days, pressure_profile / 1e5, '-r', label='Забойное давление')
    ax1.set_ylabel('Давление, бар')
    ax1.set_ylim([0, 500])
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_title('Профиль давления в скважине')
        
    ax2.plot(time_days, flow_rate_m3_day, '-g', label='Дебит')
    ax2.set_ylabel('Дебит, м³/сут')
    ax2.set_xlabel('Время, дни')
    ax2.grid(True, alpha=0.3)
    ax2.set_title('Дебит скважины во времени')
    ax2.legend()

    ax1.set_xlim([0, time_days[-1]])
    ax2.set_xlim([0, time_days[-1]])

    plt.tight_layout()
    plt.show()
    
except Exception as e:
    mr.Markdown(f"Ошибка при построении графиков: {e}")

In [ ]:
import io
import base64
from matplotlib.backends.backend_pdf import PdfPages

pdf_buffer = io.BytesIO()
with PdfPages(pdf_buffer) as pdf:
    pdf.savefig(fig, bbox_inches='tight')
pdf_buffer.seek(0)

pdf_b64 = base64.b64encode(pdf_buffer.getvalue()).decode()
pdf_download = f'<a href="data:application/pdf;base64,{pdf_b64}" download="результаты_расчета.pdf">Скачать PDF с графиками</a>'
mr.Markdown(pdf_download)